### 【[周末影音解盤](https://www.facebook.com/kevinyang20180801/videos/732373362185324/)】短線翻空，緊盯支撐  
加權指數20日線會不會下彎？第一段關鍵支撐在那邊？  
多方反攻最關鍵的時間是甚麼時候？  
未來修正的時間和空間，要看那些關鍵指標？  
完整內容請參考影音解盤~  
https://www.facebook.com/kevinyang20180801/videos/732373362185324  
  
這個周末盤勢波動大，私訊的同學較多，提早上傳解盤內容~  

### 本日進度  
* 整合[股票分析機器人](https://colab.research.google.com/drive/1gneFQWNI6PfkuE9WvN26bVMm702weH8D#scrollTo=j1x0glPsNJe2)：[最強 AI 投資分析：打造自己的股市顧問機器人，股票趨勢分析×年報解讀×選股推薦×風險管理](https://www.books.com.tw/products/0010973679)（[F3933 最強 AI 投資分析 - 服務專區](https://www.flag.com.tw/bk/t/f3933)）
* 更新「筆記」環境
  ```
  conda update -c conda-forge pip --yes
  conda update -c conda-forge numpy scipy --yes
  conda update -c conda-forge matplotlib mplfinance --yes
  conda update -c conda-forge ta-lib --yes
  conda update -c conda-forge python-dotenv --yes
  conda update -c plotly plotly --yes
  pip install pyfolio --upgrade
  pip install backtrader --upgrade
  pip install openpyxl --upgrade
  pip install FinMind --upgrade

  pip install openai
  pip install yfinance

  conda env export --name StockNotes --file StockNotes.yml
  ``` 

##### 1️⃣ 匯入套件

In [1]:
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

##### 2️⃣ 輸入 OpenAI API KEY

In [2]:
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰： ········


##### 3️⃣ 取得股價資料

In [3]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
    if stock_id == "大盤":
        stock_id="^TWII"
    else:
        stock_id += ".TW"

    end = dt.date.today() # 資料結束時間
    start = end - dt.timedelta(days=days) # 資料開始時間
    # 下載資料
    df = yf.download(stock_id, start=start)

    # 更換列名
    df.columns = ['開盤價', '最高價', '最低價',
                    '收盤價', '調整後收盤價', '成交量']

    data = {
        '日期': df.index.strftime('%Y-%m-%d').tolist(),
        '收盤價': df['收盤價'].tolist(),
        '每日報酬': df['收盤價'].pct_change().tolist(),
        '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

    return data

print(stock_price("2330"))

[*********************100%%**********************]  1 of 1 completed

{'日期': ['2023-12-28', '2023-12-29', '2024-01-02', '2024-01-03', '2024-01-04', '2024-01-05'], '收盤價': [593.0, 593.0, 593.0, 578.0, 580.0, 576.0], '每日報酬': [nan, 0.0, 0.0, -0.025295109612141653, 0.0034602076124568004, -0.006896551724137945], '漲跌價差': [nan, 0.0, 0.0, -15.0, 2.0, -4.0]}


##### 4️⃣ 取得基本面資料

In [4]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
    if stock_id == "大盤":
        return None

    stock_id += ".TW"
    stock = yf.Ticker(stock_id)

    # 營收成長率
    quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

    # 每季EPS
    quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

    # EPS季增率
    quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

    # 轉換日期
    dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

    data = {
        '季日期': dates[:len(quarterly_revenue_growth)],  # 以最短的數據列表長度為准，確保數據對齊
        '營收成長率': quarterly_revenue_growth.tolist(),
        'EPS': quarterly_eps.tolist(),
        'EPS 季增率': quarterly_eps_growth.tolist()
    }

    return data

print(stock_fundamental("2330"))

{'季日期': ['2023-09-30', '2023-06-30', '2023-03-31'], '營收成長率': [1136.03, -0.05, -0.19], 'EPS': [8.14, 7.01, 7.98, 11.41], 'EPS 季增率': [0.16, -0.12, -0.3]}


##### 5️⃣ 取得新聞資料

In [5]:
# 新聞資料
def stock_news(stock_name ="大盤"):
    if stock_name == "大盤":
        stock_name="台股 -盤中速報"

    data=[]
    # 取得 Json 格式資料
    json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

    # 依照格式擷取資料
    items=json_data['data']['items']
    for item in items:
        # 網址、標題和日期
        news_id = item["newsId"]
        title = item["title"]
        publish_at = item["publishAt"]
        # 使用 UTC 時間格式
        utc_time = dt.datetime.utcfromtimestamp(publish_at)
        formatted_date = utc_time.strftime('%Y-%m-%d')
        # 前往網址擷取內容
        url = requests.get(f'https://news.cnyes.com/'
                           f'news/id/{news_id}').content
        soup = BeautifulSoup(url, 'html.parser')
        p_elements=soup .find_all('p')
        # 提取段落内容
        p=''
        for paragraph in p_elements[4:]:
            p+=paragraph.get_text()
        data.append([stock_name, formatted_date ,title,p])
    return data

print(stock_news("台積電"))

[['台積電', '2024-01-07', '〈台股風向球〉多種火種到處點火 耐心等待三件大事', '元月以來，台股紅盤行情失靈，加權指數周跌 411 點，出乎市場預期，台股交易員指出，主要是因市場對聯準會 (Fed)3 月降息的預期恐落空，美元反彈，匯市高度牽動股市，不過，一周來仍有近 10 檔個股漲幅超過 20%，更有逾 30 檔漲逾 10%，多頭火種不滅，耐心等待國內總統大選結束、以及陸續登場的重要法說和 CES 展。由於投資人開始懷疑聯準會最快可能要到 6 月才會降息，市場對資金行情的滿腔期待轉為落空情緒，新台幣走貶，連動中大型電子股攻擊力道熄火。從三大法人的布局來看，外資先行撤出中大型電子股，合計賣超 295 億元，終結連 3 賣，投信則是再買超 119 億元，為連 2 買。以金額統計，外資主要賣超聯發科 (2454-TW) 近 80 億元，緯創 (3231-TW)35 億元，南電 (8046-TW) 和晶豪科 (3006-TW) 均超過 20 億元，華城 (1519-TW)19 億元，元大高股息 (0056-TW)、技嘉 (2376-TW) 和威剛 (3260-TW) 均逾 16 億元，英業達 (2356-TW)、復華台灣科技優息 (00929-TW) 和台達電 (2308-TW) 均超過 15 億元。外資買超多為公司債，包括國泰投資級公司債和智原 (3035-TW) 均逾 53 億元，復華公司債 43 億元，富邦金融投等債、永豐 10 年 A 公司債 37 億元上，元大投資級公司債、群益 AAA-AA 公司債和華碩 (2357-TW)30 億元之上。投信買超最多者為長榮 (2603-TW) 近 32 億元，中美晶 (5483-TW)、聯詠 (3034-TW)、中信金 (2891-TW)、台積電 (2330-TW)、瑞儀 (6176-TW) 均逾 10 億元，瑞昱 (2379-TW) 和聯電 (2303-TW) 也在 9 億元上下，顯示投信反而有意逢低提前布局中大型電子股。本周重大活動為 1 月 9 日登場的 CES 展，英特爾稱今年為 AI PC 元年，預期相關 AI PC、AI 伺服器、智慧醫療、虛擬實境等業者將大鳴大放新展品的願景，AI 浪潮有望重回盤面焦點，法人樂觀看待，長期來看 AI 指標股拉回就是買點，惟短線波動還是看資金行情的拉鋸。永豐投

##### 6️⃣ 爬取股號、股名對照表

In [6]:
# 取得全部股票的股號、股名
def stock_name():
      print("線上讀取股號、股名、及產業別")

      response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
      url_data = BeautifulSoup(response.text, 'html.parser')
      stock_company = url_data.find_all('tr')

      # 資料處理
      data = [
          (row.find_all('td')[0].text.split('\u3000')[0].strip(),
           row.find_all('td')[0].text.split('\u3000')[1],
           row.find_all('td')[4].text.strip())
          for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
      ]

      df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])

      return df

name_df = stock_name()

線上讀取股號、股名、及產業別


##### 7️⃣ 取得股票名稱

In [7]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    return name_df.set_index('股號').loc[stock_id, '股名']

print(name_df.head())
print("--------------------------")
print(get_stock_name("2501",name_df))

     股號  股名   產業別
0  1101  台泥  水泥工業
1  1102  亞泥  水泥工業
2  1103  嘉泥  水泥工業
3  1104  環泥  水泥工業
4  1108  幸福  水泥工業
--------------------------
國建


##### 8️⃣ 建構 GPT 3.5 模型

In [8]:
# 建立 GPT 3.5-16k 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-1106",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


##### 9️⃣ 大盤趨勢報告

In [9]:
reply = stock_gpt(stock_id="大盤")
print(reply)

[*********************100%%**********************]  1 of 1 completed


### 股市趨勢分析報告

#### 近期股價走勢
根據提供的台灣大盤收盤價資訊，自 2023 年 12 月 28 日至 2024 年 01 月 05 日，加權指數呈現逐日下跌的趨勢。收盤價從 17910.37 點下降至 17519.14 點，約跌幅 2.18%。每日報酬率的變化也呈現近日下跌的趨勢，反映出大盤整體走勢的弱勢。

#### 新聞資訊分析
從近期新聞資訊來看，台股市場受到多方面因素影響。包括全球市場的不確定性、外資資金流向的變化、總統大選與重要法說等事件。

在外資方面，從 2023 年 11 月至 12 月出現了進入史上第一大金額的淨匯入，變更為於 2024 年 01 月出現了處於進入史上第三大金額的淨匯入，這代表外資對於台灣股市的資金流入趨勢較為積極。此外，投信的布局方面，則顯示出對中大型電子股的提前布局。

在資金面，從最新的資本流向分析顯示，市場對聯準會降息預期的轉變、外資對台股投資信心升溫等都成為影響股市的要素。此外，重要法說活動以及國際影響中最大的消費性電子 CES 展也成為市場關注的重點。

在個股方面，一些概念股、蘋果、台積電、聯發科等廠商的動向，以及與 AI PC 相關的概念股都受到市場矚目。

群益台灣精選高息 (00919-TW) 成為第 5 檔台股千億 ETF。此外，更有許多概念股的發展，如神盾 (6462-TW) 推出全新攝像頭解決方案、微軟的 Copilot 鍵以及祥碩 (5269-TW) 等 IC 設計公司的股票變化等。

#### 基本面分析
從基本面上看，近期財團法人資金的佈局、營收狀況、收益率等因素都以及市場預期的績效，都可以影響市場行情的波動。同時，有利於投資人評估收益能力和風險。

#### 投資策略建議
根據以上分析，建議投資者在選股和短期操作上需謹慎，應該關注市場走勢及消息面的變化，特別需要留意個別概念股的動向和資本流向變化。值得注意的是，對市場的評估和分析是迅速和準確的，同時建議量力而為，存在風險意識上限。

在長期投資方面，可以考慮選擇專業基金或 ETF 等進行股票配置，以分散風險。

總的來說，台股市場目前面臨著多重不確定因素，投資者應當謹慎應對，突出風險管理，同時保持敏感的投資眼光。


##### 🔟 個股分析報告

In [10]:
reply = stock_gpt(stock_id="2330")
print(reply)

[*********************100%%**********************]  1 of 1 completed


根據所提供的資訊，以下是對於台積電近期趨勢的專業分析報告：

**股價趨勢分析：**
從最近的股價資訊來看，台積電股價在過去一周內呈現一定程度的下跌。從2023年12月28日至2024年1月5日，股價由593元下跌至576元，總體呈現下行趨勢。此外，每日報酬率也顯示出股價的波動，且存在較大的每日漲跌價差，顯示市場交易活躍度較高。

**基本面分析：**
從季度營收資訊來看，台積電的營收成長率在2023年度呈現出顯著的增長，尤其是在2023年第三季度，營收成長率達到了1136.03%。然而，在2023年第二季度和第一季度，營收成長率呈現微幅下降的趨勢。這可能與市場對於半導體行業的需求有關，值得進一步觀察。

從台積電的EPS資訊來看，從2023年第一季度到第三季度，EPS呈現出些微波動，但整體依然維持在較高水平。此外，季度EPS的季增率呈現出一定的波動，但整體趨勢較為穩定。

**新聞資訊影響分析：**
最近的新聞資訊顯示，Fed降息預期的改變對台積電及台股產生了影響，而韓國三星推出AI手機也凸顯了全球AI手機市場的潛力。同時，台電強韌電網計畫持續釋單，也給台積電的供應鏈帶來了一定的積極影響。

**結論及建議：**
綜上所述，台積電目前股價呈現下行趨勢，但在基本面及新聞面影響下，其潛在價值仍可期待。建議投資者應關注行業趨勢的變化以及全球半導體市場的發展，並對投資決策審慎評估。同時，對於台積電的長期發展，值得密切關注其EPS趨勢及產業新聞資訊，以提高投資決策的準確性。

以上報告僅供參考，建議投資者在做出任何投資決策前，仍應諮詢專業意見並自行評估風險。

希望本報告對您有所幫助，如果有任何進一步的需求或疑問，請隨時告知。


* 程式碼改動：解決**「國建(2501)」**新聞資訊問題

In [11]:
# 建立 GPT 3.5-16k 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo-1106",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_id) # 改動 ： stock_name → stock_id

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data

In [12]:
reply = stock_gpt(stock_id="2501")
print(reply)

[*********************100%%**********************]  1 of 1 completed


報告

一、股價趨勢分析：
  近期股價表現如下：
  - 2023-12-28 收盤價：17.7
  - 2023-12-29 收盤價：18.15，每日報酬率為2.54%
  - 2024-01-02 收盤價：18.6，每日報酬率為2.48%
  - 2024-01-03 收盤價：18.25，每日報酬率為-1.88%
  - 2024-01-04 收盤價：18.4，每日報酬率為0.82%
  - 2024-01-05 收盤價：18.8，每日報酬率為2.17%

  股價在近期呈現波動上漲的趨勢，從12月29日至1月5日的連續上漲可以觀察到市場對公司股票的積極態度。隨著收盤價持續上揚，投資人對公司前景的樂觀情緒也在提升。

二、基本面分析：
  1. 每季營收資訊顯示公司營收成長率在過去三季有所波動，但整體表現仍呈現一定的穩定性，需要關注未來營收的動向。
  2. 公司近三季的每股盈餘（EPS）呈現盈利，並且在季增率上也出現了起伏，顯示公司盈利狀況較為穩健，但也需留意近期的盈利波動情況。

三、新聞資訊分析：
  1. 公司取得高雄市三民區營建用地，這樣的舉措可能影響到公司未來的營運規模與範疇，值得密切關注。這也代表公司在未來有可能有新的業務機會。
  2. 子公司與非關係人簽訂增補契約，提示公司擴展業務的發展意圖。這可能對公司業績產生一定影響，需要進一步觀察。
  3. 公司向關係人取得使用權資產，此舉可能與公司的未來營運或資產配置有關。
  4. 公司更正了前期的財務報告，此舉對公司的財務透明度與投資者信心可能帶來一定程度的影響。

綜合上述信息，公司股價近期持續上升，與新聞資訊及基本面有一定關聯。作為證券分析師，建議投資者應詳細研究公司的業務發展情況，並關注未來營收和盈利的趨勢變化。並建議根據自己的投資目標、風險承受能力，謹慎評估是否進行投資操作。
